# Analyse of product delivery service

Service is available on ios and android
we recieve AppMetrica, each quarter we analyse users and efficiency of attraction channels.  

There are data from AppMetrica: 1 January till 31 March 2020, 
only with users registrated after 1 January 2020.

We did analyse data of product delivery service for 3 month. 
We have some insights: 
    1. MAU for February = 75032
    2. Amount of installs in January = 80297 installations
    3. The highest convertion from installations to purchase during 7 dayes was for installations on 2020-01-01
    4. The biggest part of new users came from yandex-direct (19%).
       And the lowest from: referal (6%). 
    5. The channel with the lowest convertion to first purchase: Yandex (28%).
       And the highest cr: facebook (34%)
    6. The channel with highest sum of purchasing (median) for first purchase of users:referal (395.5). 
       And the lowest: facebook (398)
    7. We have the lowest cr step-to-step: tap_basket (24%) other around (73-78%).
    8. The highest ROMI from channel: VK (71%)
       And the lowest: google (22%). 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

In [2]:
df=pd.read_csv('KC_case_data .csv', parse_dates=['date'])

In [3]:
df.head(5)

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


###### 1. MAU for February = 75032 unique users

In [4]:
df.dtypes #we check types of data, change in first step type of date 

date            datetime64[ns]
event                   object
purchase_sum           float64
os_name                 object
device_id                int64
gender                  object
city                    object
utm_source              object
dtype: object

In [5]:
df_MAU=df.query('date>="2020-02-01" and date<"2020-03-01"').device_id.nunique()
df_MAU

75032

###### 2. Amount of installs in January = 80297 installations

In [6]:
df_installs_01=df.query('date>="2020-01-01" and date<"2020-02-01" and event=="app_install"').device_id.nunique()
df_installs_01

80297

###### 3. Calculation of convertion rate depends on day of instalation. The highest convertion was for installations on 2020-01-01.
We calculate convertion only for purchases which were made up to 7 days after instalation.

In [7]:
df_install=df.query('event=="app_install"')\
             .rename(columns={'date':'date_install', 'event':'install'})

In [8]:
df_purchase=df.query('event=="purchase"')\
              .rename(columns={'date':'date_purchase', 'event':'purchase'})\
              .groupby('device_id', as_index=False)\
              .agg({'date_purchase':'min'})

In [9]:
df_cr=pd.merge(df_install, df_purchase, on=['device_id'])

In [10]:
df_cr['convert_day']=df_cr.date_purchase-df_cr.date_install

In [11]:
df_all_install=df_install.groupby('date_install', as_index=False).agg({'install':'count'})

In [12]:
df_7days_convert=df_cr.query('convert_day>="0 days" and convert_day<="7 days"')\
                      .groupby('date_install', as_index=False).convert_day.count()\
                      .rename(columns={'convert_day':'conversion'})

In [13]:
df_cr_final=pd.merge(df_7days_convert, df_all_install, on='date_install')

In [14]:
df_cr_final['cr']=(df_cr_final.conversion/df_cr_final.install).mul(100).round(1)
df_cr_final.head(3)

,date_install,conversion,install,cr
0,2020-01-01,1408,3579,39.3
1,2020-01-02,1186,3144,37.7
2,2020-01-03,834,2402,34.7


###### 4.  The biggest part of new users came from yandex-direct (19%).

In [15]:
df_CTR=df.query('event=="app_install"')\
         .groupby(['utm_source'], as_index=False)\
         .device_id.count()\
         .rename(columns={'device_id':'CTR'})\
         .sort_values('CTR', ascending=False)
df_CTR['%']=df_CTR.CTR/df_CTR.CTR.sum()
df_CTR

,utm_source,CTR,%
0,-,32460,0.209965
6,yandex-direct,29368,0.189965
2,google_ads,26286,0.170029
5,vk_ads,23189,0.149996
3,instagram_ads,20096,0.129990
1,facebook_ads,13916,0.090015
4,referal,9282,0.060040


###### 5. Find the channel with the lowest convertion to first purchase: Yandex
CR = amount of users who did first purchase/amount of users who open app first time

In [16]:
df.head(4)

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads


In [17]:
df_start_utm=df.query('event=="app_start"')\
               .rename(columns={'date':'date_start'})\
               .groupby('utm_source', as_index=False)\
               .agg({'date_start':'min', 'device_id':'nunique'})\
               .rename(columns={'device_id':'first_opens'})
df_start_utm

,utm_source,date_start,first_opens
0,-,2020-01-01,52268
1,facebook_ads,2020-01-01,25957
2,google_ads,2020-01-01,38092
3,instagram_ads,2020-01-01,31045
4,referal,2020-01-01,15924
5,vk_ads,2020-01-01,34079
6,yandex-direct,2020-01-01,40707


In [18]:
df_purchase_utm=df.query('event=="purchase"')\
                  .groupby(['utm_source', 'device_id'], as_index=False)\
                  .agg({'device_id':'count'})\
                  .query('device_id==1')\
                  .groupby('utm_source', as_index=False)\
                  .agg({'device_id':'sum'})\
                  .rename(columns={'device_id':'sum_users_purchase'})
df_purchase_utm

,utm_source,sum_users_purchase
0,-,12224
1,facebook_ads,6421
2,google_ads,8960
3,instagram_ads,7824
4,referal,4363
5,vk_ads,9040
6,yandex-direct,9603


In [19]:
df_open_utm=df.query('event=="app_start"')\
              .groupby(['utm_source', 'device_id'], as_index=False)\
              .agg({'device_id':'count'})\
              .query('device_id==1')\
              .groupby('utm_source', as_index=False)\
              .agg({'device_id':'sum'})\
              .rename(columns={'device_id':'sum_users_open'})        
                    
df_open_utm

,utm_source,sum_users_open
0,-,39826
1,facebook_ads,19040
2,google_ads,31094
3,instagram_ads,24159
4,referal,11734
5,vk_ads,26892
6,yandex-direct,33812


In [20]:
df_cr_channels=pd.merge(df_open_utm, df_purchase_utm, on='utm_source' )
df_cr_channels

,utm_source,sum_users_open,sum_users_purchase
0,-,39826,12224
1,facebook_ads,19040,6421
2,google_ads,31094,8960
3,instagram_ads,24159,7824
4,referal,11734,4363
5,vk_ads,26892,9040
6,yandex-direct,33812,9603


In [21]:
df_cr_channels['cr']=df_cr_channels.sum_users_purchase/df_cr_channels.sum_users_open
df_cr_channels

,utm_source,sum_users_open,sum_users_purchase,cr
0,-,39826,12224,0.306935
1,facebook_ads,19040,6421,0.337237
2,google_ads,31094,8960,0.288158
3,instagram_ads,24159,7824,0.323854
4,referal,11734,4363,0.371825
5,vk_ads,26892,9040,0.336159
6,yandex-direct,33812,9603,0.284012


###### 6. Channels, where users with highest sum of purchasing (median) for first purchase: referal.

In [22]:
df_purchase=df.query('event=="purchase"')\
              .groupby('device_id', as_index=False)\
              .agg({'date':'min'})

In [23]:
df_purchase_sum=df.query('event=="purchase"')

In [24]:
df_purchase_meidan=pd.merge(df_purchase, df_purchase_sum, on=['device_id', 'date'], how='left')

In [25]:
df_purchase_sum_meidan=df_purchase_meidan.groupby(['utm_source'], as_index=False)\
                                         .agg({'purchase_sum':'median'})\
                                         .sort_values(by='purchase_sum')
df_purchase_sum_meidan

,utm_source,purchase_sum
1,facebook_ads,389.0
2,google_ads,390.5
6,yandex-direct,392.5
5,vk_ads,393.0
3,instagram_ads,393.5
4,referal,395.5
0,-,398.5


###### 7. Analyse on which step we have the lowest cr step-to-step: tap_basket

In [61]:
#we check in events just event==register and event==register or event==purchase to control do we have all registrated id in frame. 
#as we know user cannot do purchase without registration. if we do event==register, we have all id who registrated. it is enough.
#we select registrated users and merge with df to choose all other their actions.
df_registrated=df.query('event=="register"')\
                 .groupby(['device_id'], as_index=False)\
                 .agg({'gender':'count'})\
                 .merge(df, on='device_id', how='left')
df_registrated.shape

(2205966, 9)

In [77]:
df_event_to_event_cr=df_registrated.groupby(['event'], as_index=False)\
                                   .agg({'device_id':'count'})\
                                   .rename(columns={'device_id':'sum_id'})\
                                   .query('event== "search" | event== "choose_item" | event== "tap_basket" | event== "register"')\
                                   .sort_values(by='sum_id', ascending=False)

In [76]:
df_event_to_event_cr['cr']=df_event_to_event_cr.sum_id/df_event_to_event_cr.sum_id.shift(1)
df_event_to_event_cr

,event,sum_id,cr
5,search,565271,NaN
2,choose_item,440467,0.779214
6,tap_basket,322287,0.731694
4,register,78310,0.242982


###### 8. The highest ROMI from channel: VK

In [29]:
df_utm_revenue=df.groupby(['utm_source'], as_index=False).agg({'purchase_sum':'sum'})
df_utm_revenue

,utm_source,purchase_sum
0,-,21449749.5
1,facebook_ads,12249901.0
2,google_ads,12868276.0
3,instagram_ads,14546969.0
4,referal,8837044.5
5,vk_ads,16389652.5
6,yandex-direct,13915368.0


In [30]:
df_utm_revenue['marketing_cost'] = [0, 8590498, 10534878, 8561626, 0, 9553531, 10491707]

In [31]:
df_utm_revenue['ROMI']=((df_utm_revenue.purchase_sum-df_utm_revenue.marketing_cost)/df_utm_revenue.marketing_cost)\
                        .mul(100).round(2)
df_utm_revenue.sort_values(by='ROMI')

,utm_source,purchase_sum,marketing_cost,ROMI
2,google_ads,12868276.0,10534878,22.15
6,yandex-direct,13915368.0,10491707,32.63
1,facebook_ads,12249901.0,8590498,42.60
3,instagram_ads,14546969.0,8561626,69.91
5,vk_ads,16389652.5,9553531,71.56
0,-,21449749.5,0,inf
4,referal,8837044.5,0,inf
